In [2]:
from astroquery.alma import Alma

In [3]:
alma = Alma()

In [4]:
alma.login('keflavich')

INFO: Authenticating keflavich on asa.alma.cl ... [astroquery.alma.core]
INFO: Authentication successful! [astroquery.alma.core]


In [5]:
#uid://A001/X1465/X2dfe

In [6]:
import os, shutil

In [7]:
tgtdir = '/orange/adamginsburg/salt/s255ir'
if not os.path.exists(tgtdir):
    os.mkdir(tgtdir)
alma.cache_location = tgtdir
os.chdir(tgtdir)

In [8]:
alma.query({'project_code': '2019.1.00492.S'}, public=False)

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,pol_states,o_ucd,access_url,access_format,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,sensitivity_10kms,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified
,,,,,,,,deg,deg,deg,deg,deg,d,d,s,s,m,m,,,,,,,,deg,deg,,m,GHz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mJy / beam,mm,,,,,,,,arcsec,,,,,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,str64,str35,str72,str9,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,object
ADS/JAO.ALMA#2019.1.00492.S,ALMA,JAO,ALMA,uid://A001/X1465/X2dfe,cube,2,S255IR-SMA1,93.2250416666483,17.989738888889704,0.007152270575276689,Circle ICRS 93.225042 17.989739 0.003576,0.02207331938769823,59482.406104,59482.449689,2086.56,2086.56,0.001273344904877368,0.0012835611994048905,206904.4306406033,/XX/YY/,phot.flux.density;phys.polarization,http://almascience.org/aq?member_ous_id=uid://A001/X1465/X2dfe,text/html,2019.1.00492.S,Proprietary,192.6003204662394,-0.048035311474175675,6,265.57449915523875,1875000000.0,A027:DV03 A035:DA43 A089:DA52 A091:DA61 A094:DV14 A096:DA44 A101:DV08 A104:DA45 A108:DA50 A111:DV19 A115:DA64 A118:DA58 A120:DV12 A122:DA55 A123:DA63 A125:DA49 A127:DV01 A128:DV24 A129:DV02 A130:DV06 A131:DA65 A132:DA46 A133:DV25 A135:DV10 P401:DA47 P404:DV20 P408:DV17 P409:DA59 P410:DV21 P411:DA56 P412:DV22 S301:DA54 S303:DA60 S306:DA48 S307:DA53 S309:DV11 W202:DV23 W203:DA42 W207:DA51 W210:DA62,F,2022-12-04T18:25:24.000,0.02207331938769823,"[216.86..218.74GHz,1128.91kHz,540.4uJy/beam@10km/s,33.6uJy/beam@native, XX YY] U [219.56..221.44GHz,1128.91kHz,538.5uJy/beam@10km/s,33.7uJy/beam@native, XX YY] U [231.56..233.44GHz,1128.91kHz,544.3uJy/beam@10km/s,35uJy/beam@native, XX YY] U [233.56..235.44GHz,1128.91kHz,563.3uJy/beam@10km/s,36.4uJy/beam@native, XX YY]",226.1500025701628,1448.941702561918,"Ginsburg, Adam",,,,,--,"Stars' masses decide their fates. The masses of forming high-mass stars are challenging to measure. The most direct and accurate method to measure a protostar's mass is by measuring the rotation curve of its orbiting disk. However, most high-mass stars are surrounded by a dust and molecule-rich medium that obscures the disk's kinematics. We recently discovered that several molecules, NaCl and KCl, uniquely trace the disk around high-mass protostars and are not present in the obscuring gas cores. We propose to observe a sample of high-mass protostars in these lines to kinematically measure their masses and demonstrate the viability of these lines as kinematic mass tracers.",S255IR-S_a_06_TM1,"Goddi, Ciriaco; Bally, John; McGuire, Brett; Maud, Luke; Plambeck, Richard; wright, melvyn;",0.5632974347742552,0.01731824203981878,0.44342768,uid://A001/X1465/X2dfd,uid://A001/X1465/X2dfe,uid://A002/Xf0fd41/X5465,Weighing massive protostars with salts,S,TARGET,T,0.4151830629854785,T,,High-mass star formation,ISM and star formation,2021-09-30T16:34:41.133
ADS/JAO.ALMA#2019.1.00492.S,ALMA,JAO,ALMA,uid://A001/X1465/X2dfe,cube,2,S255IR-SMA1,93.2250416666483,17.989738888889704,0.007152270575276689,Circle ICRS 93.225042 17.989739 0.003576,0.02207331938769823,59482.

In [9]:
alma.retrieve_data_from_uid('uid://A001/X1465/X2dfe')

INFO: Downloading files of size 91.225313762 Gbyte... [astroquery.alma.core]
INFO: Found cached file /orange/adamginsburg/salt/s255ir/member.uid___A001_X1465_X2dfe.README.txt with expected size 3554. [astroquery.query]
INFO: Found cached file /orange/adamginsburg/salt/s255ir/2019.1.00492.S_uid___A001_X1465_X2dfe_001_of_001.tar with expected size 8389576704. [astroquery.query]
INFO: Found cached file /orange/adamginsburg/salt/s255ir/2019.1.00492.S_uid___A001_X1465_X2dfe_auxiliary.tar with expected size 198432768. [astroquery.query]
INFO: Found cached file /orange/adamginsburg/salt/s255ir/2019.1.00492.S_uid___A002_Xf0fd41_X5465.asdm.sdm.tar with expected size 82637300736. [astroquery.query]


['/orange/adamginsburg/salt/s255ir/member.uid___A001_X1465_X2dfe.README.txt',
 '/orange/adamginsburg/salt/s255ir/2019.1.00492.S_uid___A001_X1465_X2dfe_001_of_001.tar',
 '/orange/adamginsburg/salt/s255ir/2019.1.00492.S_uid___A001_X1465_X2dfe_auxiliary.tar',
 '/orange/adamginsburg/salt/s255ir/2019.1.00492.S_uid___A002_Xf0fd41_X5465.asdm.sdm.tar']

In [10]:
ls

015eef9855870df77fcf847bc7d258d86f5bdb1a1eb3420f4008372a.pickle
0269becc8ca5945ca05513fea2afad6a6ee9058a8886bab0215a8b7e.pickle
0450dbf0fe4f43007dd5952ad480245e73a707637b4ab9930a423a79.pickle
0b6ea584d0b533f96ec15b197c95bddcef83e18d5e7af38ad1b3622a.pickle
0caa81c319484000884e292c6957dd90fe21464b3a0e1ccf0a445162.pickle
0da80578b6903373fadd779464bbdd064d688be631c265765b52d4f2.pickle
0e1ba0dbebd3aa9450efa1b376aad1bccb974cbf4984f93aab554f31.pickle
0fbbee9884224582c13597a9411529f55fac2e00273657eb5d1e4c4e.pickle
10bc994f46efb88bf448523d4b986c62be06c9a61659743c7823449f.pickle
144ac106b8958d89b2a9fea5299b30ad1590ddb96ca8d7321dd0f8f4.pickle
1533c97bdd94e358ff6a57cca297be99a997685affd92df1005dde91.pickle
17f2f4ab393f6a0ce2931887f6e80585bc2bacbc9b16cd4561159f2b.pickle
196c4ca22c3a97513840db2ce7190babbdd703af2508a696ff419402.pickle
2013.1.01365.S_uid___A001_X12f_X9f_auxiliary.tar
2019.1.00492.S_uid___A001_X1465_X2dfe_001_of_001.tar
2019.1.00492.S_uid___A001_X1465_X2dfe_auxiliary.tar
2019.1.00492.S

In [11]:
import glob, tarfile

In [12]:
for fn in glob.glob("*.tar"):
    with tarfile.Tarfile.open(fn) as tf:
        tf.extractall()

NameError: name 'tarfile' is not defined